In [1]:
%pip install numpy==1.26.4 pandas scikit-learn xgboost lightgbm catboost torch

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement xgboost (from versions: none)
ERROR: No matching distribution found for xgboost


In [1]:
import os
from smolagents import CodeAgent

from smolagents import OpenAIServerModel
from dotenv import load_dotenv
load_dotenv()

d:\Mephi\22kaf\4sem\AgentSystem\LangChain\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
from phoenix.otel import register
from openinference.instrumentation.smolagents import SmolagentsInstrumentor

register()
SmolagentsInstrumentor().instrument()

OpenTelemetry Tracing Details
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [3]:
model = OpenAIServerModel(
    model_id="gpt-4o-mini",
    api_base="https://api.openai.com/v1/",
    api_key=os.environ["OPENAI_API_KEY"],
    temperature = 0.0,
)

In [4]:
agent_MLE = CodeAgent(
    tools = [],
    model = model,
    additional_authorized_imports=['numpy', 'pandas', 'sklearn', 'xgboost', 'lightgbm', 'catboost', 'torch'],
    #planning_interval=4
)

In [5]:
MLE_AGENT_PROMPT = ''' You are an experienced Machine Learning Engineer (MLE) specializing in code generation, data analysis, model tuning, and solving machine learning problems. All answers must be technically accurate, concise, and consistent with best practices in ML. You will be given a dataset and a task to solve as best you can.

To do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.
To solve the task, you must plan forward to proceed in a series of steps, in a cycle of "Thought:", "Code:", and "Observation:" sequences.

At each step, in the "Thought:" sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.
Then in the "Code:" sequence, you should write the code in simple Python. The code sequence must end with the `<end_code>` sequence.
During each intermediate step, you can use `print()` to save whatever important information you will then need. These print outputs will then appear in the "Observation:" field, which will be available as input for the next step.
In the end, you have to return a final answer using the `final_answer` tool.

Here are a few examples using notional tools:
---
Task: "Using numpy, compute eigenvalues of a 3x3 symmetric matrix."

Thought: Utilize numpy's linear algebra module for eigenvalue decomposition.  
Code:
```py
import numpy as np

matrix = np.array([[2, -1, 0], [-1, 2, -1], [0, -1, 2]])
eigenvalues = np.linalg.eigh(matrix)[0]
final_answer(eigenvalues.round(2))
```<end_code>

---
Task: "With pandas, resample daily temperature data to monthly averages with forward fill."

Thought: Apply pandas resample and interpolation methods.  
Code:
```py
import pandas as pd
df = load_dataset("temperature_data")
df.index = pd.to_datetime(df["date"])
monthly_avg = df["temp"].resample("M").mean().ffill()
final_answer(monthly_avg.tail(6))
```<end_code>

---
Task: "Train sklearn RandomForestClassifier with GridSearchCV on breast_cancer dataset."

Thought: Configure hyperparameter tuning with cross-validation.  
Code:
```py
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

X, y = load_dataset("breast_cancer").data, load_dataset("breast_cancer").target
params = {'n_estimators': [100, 200], 'max_depth': [5, 10]}
grid = GridSearchCV(RandomForestClassifier(), param_grid=params, cv=3)
grid.fit(X, y)
final_answer(grid.best_params_)
```<end_code>

---
Task: "Optimize XGBoost regression on diabetes dataset using MAE metric."

Thought: Configure custom objective and evaluation metric.  
Code:
```py
from xgboost import XGBRegressor
from sklearn.datasets import load_diabetes

X, y = load_diabetes(return_X_y=True)
model = XGBRegressor(objective='reg:absoluteerror', eval_metric='mae')
model.fit(X, y)
final_answer(model.feature_importances_.round(3))
```<end_code>

---
Task: "Implement LightGBM for multiclass classification on iris with categorical features."

Thought: Explicitly handle categorical columns in LightGBM.  
Code:
```py
import lightgbm as lgb
from sklearn.datasets import load_iris

X, y = load_iris(return_X_y=True)
categoricals = [0]  # Assume first feature is categorical
model = lgb.LGBMClassifier()
model.fit(X, y, categorical_feature=categoricals)
final_answer(model.score(X, y))
```<end_code>

---
Task: "Train CatBoostRanker on MSLR-WEB10K dataset with pairwise loss."

Thought: Configure specialized ranking parameters.  
Code:
```py
from catboost import CatBoostRanker

train_pool = load_dataset("mslr_web10k_train")
model = CatBoostRanker(loss_function="PairLogit", iterations=500)
model.fit(train_pool)
final_answer(model.get_best_iteration())
```<end_code>

---
Task: "Implement PyTorch LSTM for time series forecasting with teacher forcing."

Thought: Design recurrent architecture with dropout layers.  
Code:
```py
import torch.nn as nn

class LSTMForecaster(nn.Module):
    def __init__(self, input_size=1, hidden_size=50):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(0.2)
        self.linear = nn.Linear(hidden_size, 1)
        
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.dropout(x)
        return self.linear(x[:,-1,:])

final_answer(LSTMForecaster())
```<end_code>

Above examples were using notional tools that might not exist for you. On top of performing computations in the Python code snippets that you create, you only have access to these tools:
{%- for tool in tools.values() %}
- {{ tool.name }}: {{ tool.description }}
    Takes inputs: {{ tool.inputs }}
    Returns an output of type: {{ tool.output_type }}
{%- endfor %}

{%- if managed_agents and managed_agents.values() | list %}
You can also give tasks to team members.
Calling a team member works the same as for calling a tool: simply, the only argument you can give in the call is "task", a long string explaining your task.
Given that this team member is a real human, you should be very verbose in your task.
Here is a list of the team members that you can call:
{%- for agent in managed_agents.values() %}
- {{ agent.name }}: {{ agent.description }}
{%- endfor %}
{%- endif %}

Here are the rules you should always follow to solve your task:
1. Always provide a 'Thought:' sequence, and a 'Code:\n```py' sequence ending with '```<end_code>' sequence, else you will fail.
2. Use only variables that you have defined!
3. Always use the right arguments for the tools. DO NOT pass the arguments as a dict as in 'answer = wiki({'query': "What is the place where James Bond lives?"})', but use the arguments directly as in 'answer = wiki(query="What is the place where James Bond lives?")'.
4. Take care to not chain too many sequential tool calls in the same code block, especially when the output format is unpredictable. For instance, a call to search has an unpredictable return format, so do not have another tool call that depends on its output in the same block: rather output results with print() to use them in the next block.
5. Call a tool only when needed, and never re-do a tool call that you previously did with the exact same parameters.
6. Don't name any new variable with the same name as a tool: for instance don't name a variable 'final_answer'.
7. Never create any notional variables in our code, as having these in your logs will derail you from the true variables.
8. You can use imports in your code, but only from the following list of modules: {{authorized_imports}}
9. The state persists between code executions: so if in one step you've created variables or imported modules, these will all persist.
10. Don't give up! You're in charge of solving the task, not providing directions to solve it.

Now Begin! If you solve the task correctly, you will receive a reward of $1,000,000.
'''


In [6]:
MLE_AGENT_PROMPT2 = ''' You are an experienced Machine Learning Engineer (MLE) specializing in code generation, data analysis, model tuning, and solving machine learning problems. All answers must be technically accurate, concise, and consistent with best practices in ML. You will be given a dataset and a task to solve as best you can.

To do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.
To solve the task, you must plan forward to proceed in a series of steps, in a cycle of "Thought:", "Code:", and "Observation:" sequences.

At each step, in the "Thought:" sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.
Then in the "Code:" sequence, you should write the code in simple Python. The code sequence must end with the `<end_code>` sequence.
During each intermediate step, you can use `print()` to save whatever important information you will then need. These print outputs will then appear in the "Observation:" field, which will be available as input for the next step.
In the end, you have to return a final answer using the `final_answer` tool.

You have access to the following Python libraries:
NumPy:
  Provides high-performance N-dimensional array (ndarray) data structures and mathematical operations
  for efficient numerical computing in Python.

Pandas:
  Offers DataFrame and Series data structures for easy manipulation and analysis of structured tabular data
  (e.g. CSV/Excel) with rich operations for filtering, grouping, and combining.

Scikit-learn (sklearn):
  A library of classical machine learning tools with a consistent interface, offering algorithms for classification,
  regression, clustering, and preprocessing of tabular data.

XGBoost:
  An optimized gradient boosting library using decision trees, designed for speed and performance on structured data
  with features like parallel processing and regularization.

LightGBM:
  Microsoft's gradient boosting framework that uses tree-based learning with histogram binning to achieve fast training
  and low memory usage on large datasets.

CatBoost:
  Yandex's gradient boosting library for decision trees, with native support for categorical features and efficient
  GPU training, reducing the need for manual preprocessing.

PyTorch (torch):
  A deep learning framework providing GPU-accelerated tensor computations and dynamic computation graphs,
  widely used for building and training neural network models

Here are a few examples using notional tools:
---
Task: "Using numpy, compute eigenvalues of a 3x3 symmetric matrix."

Thought: Utilize numpy's linear algebra module for eigenvalue decomposition.  
Code:
```py
import numpy as np

matrix = np.array([[2, -1, 0], [-1, 2, -1], [0, -1, 2]])
eigenvalues = np.linalg.eigh(matrix)[0]
final_answer(eigenvalues.round(2))
```<end_code>

---
Task: "With pandas, resample daily temperature data to monthly averages with forward fill."

Thought: Apply pandas resample and interpolation methods.  
Code:
```py
import pandas as pd
df = load_dataset("temperature_data")
df.index = pd.to_datetime(df["date"])
monthly_avg = df["temp"].resample("M").mean().ffill()
final_answer(monthly_avg.tail(6))
```<end_code>

---
Task: "Train sklearn RandomForestClassifier with GridSearchCV on breast_cancer dataset."

Thought: Configure hyperparameter tuning with cross-validation.  
Code:
```py
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

X, y = load_dataset("breast_cancer").data, load_dataset("breast_cancer").target
params = {'n_estimators': [100, 200], 'max_depth': [5, 10]}
grid = GridSearchCV(RandomForestClassifier(), param_grid=params, cv=3)
grid.fit(X, y)
final_answer(grid.best_params_)
```<end_code>

---
Task: "Optimize XGBoost regression on diabetes dataset using MAE metric."

Thought: Configure custom objective and evaluation metric.  
Code:
```py
from xgboost import XGBRegressor
from sklearn.datasets import load_diabetes

X, y = load_diabetes(return_X_y=True)
model = XGBRegressor(objective='reg:absoluteerror', eval_metric='mae')
model.fit(X, y)
final_answer(model.feature_importances_.round(3))
```<end_code>

---
Task: "Implement LightGBM for multiclass classification on iris with categorical features."

Thought: Explicitly handle categorical columns in LightGBM.  
Code:
```py
import lightgbm as lgb
from sklearn.datasets import load_iris

X, y = load_iris(return_X_y=True)
categoricals = [0]  # Assume first feature is categorical
model = lgb.LGBMClassifier()
model.fit(X, y, categorical_feature=categoricals)
final_answer(model.score(X, y))
```<end_code>

---
Task: "Train CatBoostRanker on MSLR-WEB10K dataset with pairwise loss."

Thought: Configure specialized ranking parameters.  
Code:
```py
from catboost import CatBoostRanker

train_pool = load_dataset("mslr_web10k_train")
model = CatBoostRanker(loss_function="PairLogit", iterations=500)
model.fit(train_pool)
final_answer(model.get_best_iteration())
```<end_code>

---
Task: "Implement PyTorch LSTM for time series forecasting with teacher forcing."

Thought: Design recurrent architecture with dropout layers.  
Code:
```py
import torch.nn as nn

class LSTMForecaster(nn.Module):
    def __init__(self, input_size=1, hidden_size=50):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(0.2)
        self.linear = nn.Linear(hidden_size, 1)
        
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.dropout(x)
        return self.linear(x[:,-1,:])

final_answer(LSTMForecaster())
```<end_code>

Above examples were using notional tools that might not exist for you. On top of performing computations in the Python code snippets that you create, you only have access to these tools:
{%- for tool in tools.values() %}
- {{ tool.name }}: {{ tool.description }}
    Takes inputs: {{ tool.inputs }}
    Returns an output of type: {{ tool.output_type }}
{%- endfor %}

{%- if managed_agents and managed_agents.values() | list %}
You can also give tasks to team members.
Calling a team member works the same as for calling a tool: simply, the only argument you can give in the call is "task", a long string explaining your task.
Given that this team member is a real human, you should be very verbose in your task.
Here is a list of the team members that you can call:
{%- for agent in managed_agents.values() %}
- {{ agent.name }}: {{ agent.description }}
{%- endfor %}
{%- endif %}

Here are the rules you should always follow to solve your task:
1. Always provide a 'Thought:' sequence, and a 'Code:\n```py' sequence ending with '```<end_code>' sequence, else you will fail.
2. Use only variables that you have defined!
3. Always use the right arguments for the tools. DO NOT pass the arguments as a dict as in 'answer = wiki({'query': "What is the place where James Bond lives?"})', but use the arguments directly as in 'answer = wiki(query="What is the place where James Bond lives?")'.
4. Take care to not chain too many sequential tool calls in the same code block, especially when the output format is unpredictable. For instance, a call to search has an unpredictable return format, so do not have another tool call that depends on its output in the same block: rather output results with print() to use them in the next block.
5. Call a tool only when needed, and never re-do a tool call that you previously did with the exact same parameters.
6. Don't name any new variable with the same name as a tool: for instance don't name a variable 'final_answer'.
7. Never create any notional variables in our code, as having these in your logs will derail you from the true variables.
8. You can use imports in your code, but only from the following list of modules: {{authorized_imports}}
9. The state persists between code executions: so if in one step you've created variables or imported modules, these will all persist.
10. Don't give up! You're in charge of solving the task, not providing directions to solve it.

Now Begin! If you solve the task correctly, you will receive a reward of $1,000,000.
'''


In [7]:
agent_MLE.prompt_templates['system_prompt'] = MLE_AGENT_PROMPT2

In [8]:
print(agent_MLE.prompt_templates['planning'])

{'initial_plan': 'You are a world express at analyzing a situation to derive facts, and plan accordingly towards solving a task.\nBelow I will present you a task. You will need to 1. build a survey of facts known or needed to solve the task, then 2. make a plan of action to solve the task.\n\n1. You will build a comprehensive preparatory survey of which facts we have at our disposal and which ones we still need.\nTo do so, you will have to read the task and identify things that must be discovered in order to successfully complete it.\nDon\'t make any assumptions. For each item, provide a thorough reasoning. Here is how you will structure this survey:\n\n---\n## Facts survey\n### 1.1. Facts given in the task\nList here the specific facts given in the task that could help you (there might be nothing here).\n\n### 1.2. Facts to look up\nList here any facts that we may need to look up.\nAlso list where to find each of these, for instance a website, a file... - maybe the task contains some 

In [9]:
task_1 = '''Your task is to train a machine learning model and measure the metric on a test set (task of classification).
You are given 'train.csv' (training data) and 'test.csv' (testing data).

You can train any models from libraries and frameworks that are given to you.
You need to predict the target variable 'Churn'.
After training, evaluate the quality on a test set.
Evaluate the trained model on the test dataset using the ROC-AUC metric.
As a final answer, give: 
1. The value of the metric 
2. A description of how the data was processed and the model was trained.
'''

In [10]:
task_2 = '''Your task is to train a machine learning model and measure the metric on a test set (regression task).
You are given 'train_flat.csv' (training data) and 'test_flat.csv' (testing data).

You can train any models from libraries and frameworks that are given to you.
You need to predict the target variable 'price'.
After training, evaluate the quality on a test set.
Evaluate the trained model on the test dataset using the RMSE metric.
As a final answer, give: 
1. The value of the metric 
2. A description of how the data was processed and the model was trained.
'''

In [12]:
analyst_result = agent_MLE.run(
    task_1
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Your task is to train a machine learning model and measure the metric on a test set (task of classification).   │
│ You are given 'train.csv' (training data) and 'test.csv' (testing data).                                        │
│                                                                                                                 │
│ You can train any models from libraries and frameworks that are given to you.                                   │
│ You need to predict the target variable 'Churn'.                                                                │
│ After training, evaluate the quality on a test set.                                                             │
│ Evaluate the trained model on the test dataset using the ROC-AUC metric.                                        │
│ As a final answer, give:                                                                                        │
│ 1. The value of the metric                                                                                      │
│ 2. A description of how the data was processed and the model was trained.                                       │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o-mini ───────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
                                                                                                                   
  # Load the datasets                                                                                              
  train_df = pd.read_csv('train.csv')                                                                              
  test_df = pd.read_csv('test.csv')                                                                                
                                                                                                                   
  # Display the first few rows of the training dataset                                                             
  print(train_df.head())                                                                                           
  print(test_df.head())                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
   ClientPeriod  MonthlySpending TotalSpent  gender  IsSeniorCitizen  \
0             3             19.3       54.7  Female                0   
1            72             84.1    6129.65  Female                1   
2            72            112.1    7965.95  Female                0   
3            42            100.4    4303.65  Female                0   
4            61            118.6     7365.7  Female                0   

  HasPartner HasChild HasPhoneService HasMultiplePhoneNumbers  \
0        Yes      Yes             Yes                      No   
1        Yes       No             Yes                     Yes   
2        Yes       No             Yes                     Yes   
3        Yes      Yes             Yes                      No   
4         No       No             Yes                     Yes   

  HasInternetService HasOnlineSecurityService      HasOnlineBackup  \
0                 No      No internet service  No internet service   
1                DSL                      Yes                  Yes   
2        Fiber optic                      Yes                  Yes   
3        Fiber optic                       No                   No   
4        Fiber optic                      Yes                  Yes   

   HasDeviceProtection HasTechSupportAccess          HasOnlineTV  \
0  No internet service  No internet service  No internet service   
1                  Yes                   No                  Yes   
2                  Yes                  Yes                  Yes   
3                  Yes                  Yes                  Yes   
4                  Yes                  Yes                  Yes   

  HasMovieSubscription HasContractPhone IsBillingPaperless  \
0  No internet service   Month-to-month                 No   
1                  Yes         Two year                Yes   
2                  Yes         Two year                 No   
3                  Yes         Two year                Yes   
4                  Yes         One year                Yes   

             PaymentMethod  Churn  
0             Mailed check      0  
1  Credit card (automatic)      0  
2  Credit card (automatic)      0  
3         Electronic check      0  
4         Electronic check      0  
   ClientPeriod  MonthlySpending  IsSeniorCitizen  Churn  TotalSpent_100.2  \
0             9            45.40                0      1               NaN   
1            37            70.35                0      0               NaN   
2            50           105.95                0      1               NaN   
3            44           100.10                0      0               NaN   
4            25            20.95                0      0               NaN   

   TotalSpent_100.35  TotalSpent_100.9  TotalSpent_1001.5  TotalSpent_1003.05  \
0                NaN               NaN                NaN                 NaN   
1                NaN               NaN                NaN                 NaN   
2                NaN               NaN                NaN                 NaN   
3                NaN               NaN                NaN                 NaN   
4                NaN               NaN                NaN                 NaN   

   TotalSpent_1004.5  ...  HasOnlineTV_No internet service  HasOnlineTV_Yes  \
0                NaN  ...                            False             True   
1                NaN  ...                            False            False   
2                NaN  ...                            False             True   
3                NaN  ...                            False             True   
4                NaN  ...                             True            False   

   HasMovieSubscription_No internet service  HasMovieSubscription_Yes  \
0                                     False                      True   
1                                     False                     False   
2                                     False                      True   
3                 

[Step 1: Duration 4.71 seconds| Input tokens: 1,934 | Output tokens: 191]

...

[Step 2: Duration 16.03 seconds| Input tokens: 5,093 | Output tokens: 584]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Prepare the final answer                                                                                       
  metric_value = 0.7987058237929316                                                                                
  description = (                                                                                                  
      "The data was processed by first filling missing values using forward fill. "                                
      "Categorical variables were then one-hot encoded to convert them into a numerical format suitable for        
  modeling. "                                                                                                      
      "The training data was split into features (X) and the target variable (y), which is 'Churn'. "              
      "A RandomForestClassifier was trained on the processed training data. "                                      
      "The model was evaluated on the test set using the ROC-AUC metric, resulting in a score of approximately     
  0.799."                                                                                                          
  )                                                                                                                
                                                                                                                   
  final_answer({'answer': {'metric_value': metric_value, 'description': description}})                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: {'metric_value': 0.7987058237929316, 'description': "The data was processed by first filling 
missing values using forward fill. Categorical variables were then one-hot encoded to convert them into a numerical
format suitable for modeling. The training data was split into features (X) and the target variable (y), which is 
'Churn'. A RandomForestClassifier was trained on the processed training data. The model was evaluated on the test 
set using the ROC-AUC metric, resulting in a score of approximately 0.799."}

[Step 3: Duration 5.21 seconds| Input tokens: 8,989 | Output tokens: 801]